In [1]:
import java.nio.file.Paths
import org.apache.spark.sql.functions._
val dir_rs = Paths.get(System.getProperty("user.home"), "data", "ot", "results")
val dir_ot = Paths.get(System.getProperty("user.home"), "data", "ot", "extract")

dir_rs = /home/eczech/data/ot/results
dir_ot = /home/eczech/data/ot/extract


/home/eczech/data/ot/extract

## Load RS Data

In [2]:
var df = spark.read.parquet(dir_rs.resolve("score_association.parquet").toString)

df = [target_id: string, disease_id: string ... 3 more fields]


[target_id: string, disease_id: string ... 3 more fields]

In [3]:
df.show(3)

+---------------+--------------+-------------------+---------+-----------+
|      target_id|    disease_id|              score|is_direct| source_ids|
+---------------+--------------+-------------------+---------+-----------+
|ENSG00000000460|Orphanet_93420|             0.1817|    false|[phenodigm]|
|ENSG00000000938|   EFO_0004244|                0.1|    false|   [chembl]|
|ENSG00000000938|Orphanet_98710|0.23667500000000002|    false|[phenodigm]|
+---------------+--------------+-------------------+---------+-----------+
only showing top 3 rows



In [4]:
df.count()

1973467

In [5]:
val df_rs_as = df.select($"target_id", $"disease_id", $"score".as("score_rs"))
df_rs_as.show(3)

+---------------+--------------+-------------------+
|      target_id|    disease_id|           score_rs|
+---------------+--------------+-------------------+
|ENSG00000000460|Orphanet_93420|             0.1817|
|ENSG00000000938|   EFO_0004244|                0.1|
|ENSG00000000938|Orphanet_98710|0.23667500000000002|
+---------------+--------------+-------------------+
only showing top 3 rows



df_rs_as = [target_id: string, disease_id: string ... 1 more field]


[target_id: string, disease_id: string ... 1 more field]

In [6]:
var df = spark.read.parquet(dir_rs.resolve("score_source.parquet").toString)

df = [target_id: string, disease_id: string ... 4 more fields]


[target_id: string, disease_id: string ... 4 more fields]

In [7]:
df.show(3)

+---------------+---------------+------------------+-------------------+---------+-------------------+
|      target_id|     disease_id|         source_id|          score_raw|is_direct|              score|
+---------------+---------------+------------------+-------------------+---------+-------------------+
|ENSG00000000419|Orphanet_371047|uniprot_literature|                1.0|    false|                1.0|
|ENSG00000000457|Orphanet_206644|         phenodigm|            0.18492|    false|            0.18492|
|ENSG00000000460|Orphanet_139009|         phenodigm|0.28020652777777777|    false|0.28020652777777777|
+---------------+---------------+------------------+-------------------+---------+-------------------+
only showing top 3 rows



In [8]:
val df_rs_sc = df.select($"target_id", $"disease_id", $"source_id", $"score".as("score_rs"))
df_rs_sc.show(3)

+---------------+---------------+------------------+-------------------+
|      target_id|     disease_id|         source_id|           score_rs|
+---------------+---------------+------------------+-------------------+
|ENSG00000000419|Orphanet_371047|uniprot_literature|                1.0|
|ENSG00000000457|Orphanet_206644|         phenodigm|            0.18492|
|ENSG00000000460|Orphanet_139009|         phenodigm|0.28020652777777777|
+---------------+---------------+------------------+-------------------+
only showing top 3 rows



df_rs_sc = [target_id: string, disease_id: string ... 2 more fields]


[target_id: string, disease_id: string ... 2 more fields]

In [9]:
df_rs_sc.filter($"score_rs" <= 0).count()

0

In [10]:
df_rs_sc.count()

2291653

### Load OT Data

In [11]:
var df_ot_ev = spark.read.json(dir_ot.resolve("association.json").toString)

df_ot_ev = [disease: struct<efo_info: struct<efo_id: string, label: string ... 2 more fields>, id: string>, evidence_count: struct<datasources: struct<cancer_gene_census: double, chembl: double ... 19 more fields>, datatypes: struct<affected_pathway: double, animal_model: double ... 5 more fields> ... 1 more field> ... 7 more fields]


[disease: struct<efo_info: struct<efo_id: string, label: string ... 2 more fields>, id: string>, evidence_count: struct<datasources: struct<cancer_gene_census: double, chembl: double ... 19 more fields>, datatypes: struct<affected_pathway: double, animal_model: double ... 5 more fields> ... 1 more field> ... 7 more fields]

In [12]:
df_ot_ev.show(3)

+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+
|             disease|      evidence_count|        harmonic-sum|                  id|is_direct|                 max|             private|                 sum|              target|
+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+
|[[http://www.orph...|[[0.0, 0.0, 0.0, ...|[[0.0, 0.0, 0.0, ...|ENSG00000132383-O...|    false|[[0.0, 0.0, 0.0, ...|[[[phenodigm], [a...|[[0.0, 0.0, 0.0, ...|[[ENSG00000132383...|
|[[http://www.orph...|[[0.0, 0.0, 0.0, ...|[[0.0, 0.0, 0.0, ...|ENSG00000125551-O...|     true|[[0.0, 0.0, 0.0, ...|[[[phenodigm], [a...|[[0.0, 0.0, 0.0, ...|[[ENSG00000125551...|
|[[http://www.orph...|[[0.0, 0.0, 0.0, ...|[[0.0, 0.0, 0.0, ...|ENSG00000141756-O...|     true|[[0.0

In [56]:
df_ot_ev.count()

1973467

In [13]:
df_ot_ev.filter($"disease.id" === "EFO_0003777" && $"target.id" === "ENSG00000226777")
    .select("harmonic-sum").show(3, 1000)

+-----------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                         harmonic-sum|
+-----------------------------------------------------------------------------------------------------------------------------------------------------+
|[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0], [0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0.5]|
+-----------------------------------------------------------------------------------------------------------------------------------------------------+



In [14]:
val df_ot_as = df_ot_ev.select(
    $"target.id".as("target_id"),
    $"disease.id".as("disease_id"),
    $"harmonic-sum.overall".as("score_ot")
)
df_ot_as.show(3)

+---------------+---------------+-------------------+
|      target_id|     disease_id|           score_ot|
+---------------+---------------+-------------------+
|ENSG00000132383|Orphanet_240371|0.24183500000000002|
|ENSG00000125551|   Orphanet_974|0.24638833333333335|
|ENSG00000141756|Orphanet_314811| 0.6649999999999999|
+---------------+---------------+-------------------+
only showing top 3 rows



df_ot_as = [target_id: string, disease_id: string ... 1 more field]


[target_id: string, disease_id: string ... 1 more field]

In [15]:
df_ot_as.count()

1973467

In [16]:
import org.apache.spark.sql.Row
val to_map = udf((r: Row) => {
    r.getValuesMap[Double](r.schema.fieldNames)
})
val df_ot_sc = df_ot_ev
    .select(
        $"target.id".as("target_id"),
        $"disease.id".as("disease_id"),
        explode(to_map($"harmonic-sum.datasources"))
    )
    .withColumnRenamed("key", "source_id")
    .withColumnRenamed("value", "score_ot")
    // Exploding the struct of non-nullable doubles
    // results in a bunch of zeros that can be ignored
    .filter($"score_ot" > 0)
df_ot_sc.show(3)

+---------------+---------------+---------+-------------------+
|      target_id|     disease_id|source_id|           score_ot|
+---------------+---------------+---------+-------------------+
|ENSG00000132383|Orphanet_240371|phenodigm|0.24183500000000002|
|ENSG00000125551|   Orphanet_974|phenodigm|0.24638833333333335|
|ENSG00000141756|Orphanet_314811|      eva| 0.6649999999999999|
+---------------+---------------+---------+-------------------+
only showing top 3 rows



to_map = UserDefinedFunction(<function1>,MapType(StringType,DoubleType,false),None)
df_ot_sc = [target_id: string, disease_id: string ... 2 more fields]


[target_id: string, disease_id: string ... 2 more fields]

In [17]:
df_ot_sc.count()

2291653

## Check Source Scores

In [18]:
var df = df_ot_sc.join(df_rs_sc, Seq("target_id", "disease_id", "source_id"), "outer")
df.show(3)

+---------------+--------------+----------------+--------+--------+
|      target_id|    disease_id|       source_id|score_ot|score_rs|
+---------------+--------------+----------------+--------+--------+
|ENSG00000000460|   EFO_0009566|       phenodigm| 0.18634| 0.18634|
|ENSG00000000460| Orphanet_3337|       phenodigm| 0.18634| 0.18634|
|ENSG00000001084|Orphanet_68367|genomics_england|     1.0|     1.0|
+---------------+--------------+----------------+--------+--------+
only showing top 3 rows



df = [target_id: string, disease_id: string ... 3 more fields]


[target_id: string, disease_id: string ... 3 more fields]

In [19]:
df.printSchema()

root
 |-- target_id: string (nullable = true)
 |-- disease_id: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- score_ot: double (nullable = true)
 |-- score_rs: double (nullable = true)



In [20]:
df.count()

2291653

In [21]:
var df_na = df
    .withColumn("rs_null", $"score_rs".isNull)
    .withColumn("ot_null", $"score_ot".isNull)

df_na = [target_id: string, disease_id: string ... 5 more fields]


[target_id: string, disease_id: string ... 5 more fields]

In [22]:
df_na.groupBy("rs_null", "ot_null").count().show()

+-------+-------+-------+
|rs_null|ot_null|  count|
+-------+-------+-------+
|  false|  false|2291653|
+-------+-------+-------+



In [23]:
df_na.filter($"rs_null" && !$"ot_null").groupBy("source_id").count().show()

+---------+-----+
|source_id|count|
+---------+-----+
+---------+-----+



In [24]:
df_na.filter(!$"rs_null" && $"ot_null").groupBy("source_id").count().show()

+---------+-----+
|source_id|count|
+---------+-----+
+---------+-----+



In [25]:
var df_diff = df
    .withColumn("rs_null", $"score_rs".isNull)
    .withColumn("ot_null", $"score_ot".isNull)
    .filter(!$"rs_null" && !$"ot_null")
    .withColumn("diff", abs($"score_rs" - $"score_ot"))
    .withColumn("isequal", $"diff" < .0001)

df_diff = [target_id: string, disease_id: string ... 7 more fields]


[target_id: string, disease_id: string ... 7 more fields]

In [26]:
df_diff.groupBy($"isequal").count().show()

+-------+-------+
|isequal|  count|
+-------+-------+
|   true|2291653|
+-------+-------+



In [27]:
df_diff.groupBy($"source_id", $"isequal").count().show()

+------------------+-------+-------+
|         source_id|isequal|  count|
+------------------+-------+-------+
|uniprot_literature|   true|  40962|
|            crispr|   true|   9766|
|           intogen|   true|  12113|
|cancer_gene_census|   true|  87268|
|        slapenrich|   true| 134820|
|            chembl|   true| 114977|
|  expression_atlas|   true| 142158|
|   uniprot_somatic|   true|    319|
|  genomics_england|   true|  47859|
|         phenodigm|   true|1118373|
|           progeny|   true|   1316|
|          reactome|   true|   8369|
|      gwas_catalog|   true| 294959|
|               eva|   true|  70975|
|    phewas_catalog|   true| 136457|
|           uniprot|   true|  42549|
|            sysbio|   true|   1853|
|       eva_somatic|   true|   4810|
|    gene2phenotype|   true|  21750|
+------------------+-------+-------+



## Check Association Scores

In [28]:
var df = df_ot_as.join(df_rs_as, Seq("target_id", "disease_id"), "inner")
df.show(3)

+---------------+--------------+-------------------+-------------------+
|      target_id|    disease_id|           score_ot|           score_rs|
+---------------+--------------+-------------------+-------------------+
|ENSG00000000938|Orphanet_44890|                0.2|                0.2|
|ENSG00000000971| Orphanet_1505|0.23495000000000002|0.23495000000000002|
|ENSG00000000971|Orphanet_98687|                0.2|                0.2|
+---------------+--------------+-------------------+-------------------+
only showing top 3 rows



df = [target_id: string, disease_id: string ... 2 more fields]


[target_id: string, disease_id: string ... 2 more fields]

In [29]:
df.withColumn("isequal", abs($"score_rs" - $"score_ot") < .0001).groupBy("isequal").count().show()

+-------+-------+
|isequal|  count|
+-------+-------+
|   true|1973467|
+-------+-------+

